# Guided Exercise: Drift
This is a continuation of part 1. If you missed it:     [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/truera-examples/blob/release/rc-1.37/starter-examples/starter-drift-part-1.ipynb)

#### Goals 🎯
In this tutorial, you will learn how to:
1. View the results of stability tests set up in part 1.
2. Debug the true cause of stability issues.
3. Retest the new model and confirm the effectiveness of the mitigation strategy.

### First, set the credentials for your TruEra deployment.
If you don't have credentials yet, get them instantly by signing up for free at: https://www.truera.com

In [ ]:
#connection details
TRUERA_URL = "https://app.truera.net/"
AUTH_TOKEN = "..."

### Install the required packages for running in colab

In [ ]:
! pip install truera

### From here, run the rest of the notebook and follow the analysis.

### First, load data and train the in your beach-head market, San Francisco. Also add additional data for Seattle and Austin, your target markets.

In [ ]:
import logging
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.utils import resample

from truera.client.truera_workspace import TrueraWorkspace
from truera.client.truera_authentication import TokenAuthentication
from truera.client.ingestion import ModelOutputContext, ColumnSpec

auth = TokenAuthentication(AUTH_TOKEN)
tru = TrueraWorkspace(TRUERA_URL, auth)

### Test for stability in Seattle and Austin.

In [ ]:
# create the first project and data collection
project_name = "Starter Example Companion - Drift"
tru.set_project(project_name)
tru.set_data_collection("Data Collection v1")
tru.get_models()

In [ ]:
# add performance and feature importance tests
tru.tester.add_performance_test(
    test_name="MAE Test",
    all_data_collections=True,
    data_split_name_regex="Seattle",
    metric="MAE",
    reference_split_name="San Francisco",
    fail_if_greater_than=40,
    fail_threshold_type="RELATIVE"
)

In [ ]:
tru.tester.get_model_leaderboard(sort_by="performance")

In [ ]:
tru.set_model("model_1")
tru.tester.get_model_test_results(test_types=["stability"])

The model fails in Seattle and Austin because the scores drifted too far from the ground truth in the new cities.

In [ ]:
explainer = tru.get_explainer("Austin", comparison_data_splits=["San Francisco"])
explainer.find_hotspots(max_num_responses=5)

The MAE for larger, more expensive houses is high, as represented in the hotspots. Bedrooms are a commonly used proxy for house size and price. This will correlate with all the listed hotspots. Let's resample the San Francisco data we're training on to include an equal proportion of larger listings as Austin.

In [ ]:
# load data
san_francisco = pd.read_csv("https://truera-examples.s3.us-west-2.amazonaws.com/data/starter-stability/San_Francisco_for_stability.csv")
seattle = pd.read_csv("https://truera-examples.s3.us-west-2.amazonaws.com/data/starter-stability/Seattle_for_stability.csv")
austin = pd.read_csv("https://truera-examples.s3.us-west-2.amazonaws.com/data/starter-stability/Austin_for_stability.csv")

# make all float and make index ids
san_francisco = san_francisco.astype(float).reset_index(names="id")
seattle = seattle.astype(float).reset_index(names="id")
austin = austin.astype(float).reset_index(names="id")

In [ ]:
large_listings = san_francisco[san_francisco["bedrooms"] >= 2]
small_listings = san_francisco[san_francisco["bedrooms"] < 2]

austin_large_listings = austin[austin["bedrooms"] >= 2]
num_samples = int(round((len(austin_large_listings)/len(austin)) * len(san_francisco), 0))

large_listings_resampled = resample(
    large_listings, 
    replace=True,
    n_samples=num_samples,
    random_state=1 # include random seed so we can perform same sampling on each data set
)

san_francisco_resampled = pd.concat([small_listings, large_listings_resampled])

In [ ]:
# train new model on resampled sf data
xgb_reg = xgb.XGBRegressor(eta=0.2, max_depth=4)
xgb_reg.fit(san_francisco_resampled.drop(["id", "price"], axis=1), san_francisco_resampled.price)

tru.set_project(project_name)
tru.set_data_collection("Data Collection v1")

# register the model
tru.add_python_model(
    "model_2",
    xgb_reg,
    train_parameters={"model_type": "xgb.XGBRegressor", "eta": 0.2, "max_depth": 4}
)

In [ ]:
# predictions
tru.set_project(project_name)
tru.set_data_collection("Data Collection v1")
tru.set_model("model_2")
tru.set_influences_background_data_split("San Francisco")
tru.set_data_split("San Francisco")
sf_preds = tru.get_ys_pred().reset_index(names = "id")
tru.set_data_split("Seattle")
se_preds = tru.get_ys_pred().reset_index(names = "id")
tru.set_data_split("Austin")
au_preds = tru.get_ys_pred().reset_index(names = "id")

tru.add_data(
    data = sf_preds,
    data_split_name = "San Francisco",
    column_spec=ColumnSpec(
        id_col_name="id",
        prediction_col_names="__truera_prediction__"
    )
)

tru.add_data(
    data = se_preds,
    data_split_name = "Seattle",
    column_spec=ColumnSpec(
        id_col_name="id",
        prediction_col_names="__truera_prediction__"
    )
)

tru.add_data(
    data = au_preds,
    data_split_name = "Austin",
    column_spec=ColumnSpec(
        id_col_name="id",
        prediction_col_names="__truera_prediction__"
    )
)

In [ ]:
# set model and background split
tru.set_model("model_2")
tru.set_influences_background_data_split("San Francisco")

# influence type
tru.set_influence_type("shap")

# reduce settings for speed
tru.set_num_internal_qii_samples(100)
tru.set_num_default_influences(100)

se_explainer = tru.get_explainer("Seattle")
se_infs = se_explainer.get_feature_influences().reset_index(names = "id")

sf_explainer = tru.get_explainer("San Francisco")
sf_infs = sf_explainer.get_feature_influences().reset_index(names = "id")

au_explainer = tru.get_explainer("Austin")
au_infs = sf_explainer.get_feature_influences().reset_index(names = "id")

model_output_context = ModelOutputContext(model_name='model_2', score_type='regression', background_split_name='San Francisco', influence_type='kernel-shap')

tru.add_data(
    data = sf_infs,
    data_split_name = "San Francisco",
    column_spec = ColumnSpec(
        id_col_name = "id",
        feature_influence_col_names = list(sf_infs.columns.drop("id"))
    ),
    model_output_context=model_output_context
)

tru.add_data(
    data = se_infs,
    data_split_name = "Seattle",
    column_spec = ColumnSpec(
        id_col_name = "id",
        feature_influence_col_names = list(se_infs.columns.drop("id"))
    ),
    model_output_context=model_output_context
)

tru.add_data(
    data = au_infs,
    data_split_name = "Austin",
    column_spec = ColumnSpec(
        id_col_name = "id",
        feature_influence_col_names = list(se_infs.columns.drop("id"))
    ),
    model_output_context=model_output_context
)

In [ ]:
# error influences
model_output_context = ModelOutputContext(model_name='model_2', score_type='mean_absolute_error_for_regression', background_split_name='San Francisco', influence_type='kernel-shap')

tru.set_data_split('San Francisco')
sf_error_infs = tru.get_feature_influences(score_type='mean_absolute_error_for_regression').reset_index(names="id")

tru.set_data_split('Seattle')
se_error_infs = tru.get_feature_influences(score_type='mean_absolute_error_for_regression').reset_index(names="id")

tru.set_data_split('Austin')
au_error_infs = tru.get_feature_influences(score_type='mean_absolute_error_for_regression').reset_index(names="id")

tru.add_data(
    data = sf_error_infs,
    data_split_name = "San Francisco",
    column_spec = ColumnSpec(
        id_col_name = "id",
        feature_influence_col_names = list(sf_error_infs.columns.drop("id"))
    ),
    model_output_context=model_output_context
)

tru.add_data(
    data = se_error_infs,
    data_split_name = "Seattle",
    column_spec = ColumnSpec(
        id_col_name = "id",
        feature_influence_col_names = list(se_error_infs.columns.drop("id"))
    ),
    model_output_context=model_output_context
)

tru.add_data(
    data = au_error_infs,
    data_split_name = "Austin",
    column_spec = ColumnSpec(
        id_col_name = "id",
        feature_influence_col_names = list(se_error_infs.columns.drop("id"))
    ),
    model_output_context=model_output_context
)

In [ ]:
# check drift results
tru.set_environment("remote")
tru.set_model("model_2")
tru.tester.get_model_test_results(test_types=["stability"])

The model now passes in Austin and is ready for production, while it still fails in Seattle. Let's continue to iterate on Seattle.

Since the model errors with scores that are too high, we should look for the largest positive contributors to score drift.

In [ ]:
explainer = tru.get_explainer("San Francisco", comparison_data_splits=["Seattle"])
explainer.compute_feature_contributors_to_instability(use_difference_of_means=True).T

Availability_90 is by far the largest positive contributor to score drift in Seattle. Let's remove that feature along with the related feature Availability_365 to mitigate this issue.

In [ ]:
# train a new model
xgb_reg = xgb.XGBRegressor(eta=0.2, max_depth=4)
xgb_reg.fit(san_francisco_resampled.drop(["id", "price", "availability_90", "availability_365"], axis=1), san_francisco_resampled.price)

# create the first project and data collection
tru.add_data_collection("Data Collection v2")

In [ ]:
# add data to the collection we just created
tru.add_data(
    data = san_francisco,
    data_split_name = "San Francisco",
    column_spec=ColumnSpec(
        id_col_name = "id",
        pre_data_col_names=list(san_francisco.columns.drop(["id","price","availability_90","availability_365"])),
        label_col_names="price")
)
tru.add_data(
    data = seattle,
    data_split_name = "Seattle",
    column_spec=ColumnSpec(
        id_col_name = "id",
        pre_data_col_names=list(seattle.columns.drop(["id","price","availability_90","availability_365"])),
        label_col_names="price")
)
tru.add_data(
    data = austin,
    data_split_name = "Austin",
    column_spec=ColumnSpec(
        id_col_name = "id",
        pre_data_col_names=list(seattle.columns.drop(["id","price","availability_90","availability_365"])),
        label_col_names="price")
)
tru.set_influences_background_data_split("San Francisco")

# register the model
tru.add_python_model(
    "model_3",
    xgb_reg,
    train_parameters={"model_type": "xgb.XGBRegressor", "eta": 0.2, "max_depth": 4}
)

In [ ]:
tru.set_model('model_3')

In [ ]:
tru.set_data_split("San Francisco")
sf_preds = tru.get_ys_pred().reset_index(names = "id")
tru.set_data_split("Seattle")
se_preds = tru.get_ys_pred().reset_index(names = "id")
tru.set_data_split("Austin")
au_preds = tru.get_ys_pred().reset_index(names = "id")

In [ ]:
tru.set_data_split("San Francisco")
sf_preds = tru.get_ys_pred().reset_index(names = "id")
tru.set_data_split("Seattle")
se_preds = tru.get_ys_pred().reset_index(names = "id")
tru.set_data_split("Austin")
au_preds = tru.get_ys_pred().reset_index(names = "id")

tru.add_data(
    data = sf_preds,
    data_split_name = "San Francisco",
    column_spec=ColumnSpec(
        id_col_name="id",
        prediction_col_names="__truera_prediction__"
    )
)

tru.add_data(
    data = se_preds,
    data_split_name = "Seattle",
    column_spec=ColumnSpec(
        id_col_name="id",
        prediction_col_names="__truera_prediction__"
    )
)

tru.add_data(
    data = au_preds,
    data_split_name = "Austin",
    column_spec=ColumnSpec(
        id_col_name="id",
        prediction_col_names="__truera_prediction__"
    )
)

In [ ]:
# get the test details from model_2 so we can copy them for model_3
tru.set_environment("remote")
tru.set_model("model_2")
tru.tester.get_model_tests().as_dict()["Stability Tests"]["Rows"]

In [ ]:
# toggle back to remote to interact with the tester

# check stability results
tru.set_model("model_3")

# let the warn conditions have $50 in wiggle room
tru.tester.add_stability_test(test_name="Stability Test - Seattle - v3",
    base_data_split_name="San Francisco",
    comparison_data_split_names=["Seattle"],
    fail_if_outside=[-142.44841, -12.44841]
)


In [ ]:
tru.tester.get_model_test_results(test_types=["stability"])

In v3, the model passes now in Seattle. We can deploy the v2 model in Austin and v3 model in Seattle as we launch and the investors of our startup are satisfied with these results!